# Capstone - Segmenting and Clustering Neighbourhoods in Toronto

# Part 1.  Getting data from Wikipedia, creating and formatting data frame.

In [1]:
#import needed libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [3]:
# download data from Wikipedia, use Beautiful Soup to scrape table
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup= BeautifulSoup(source, 'xml')

In [4]:
#make soup object into a table
table=soup.find('table')

In [5]:
#make the dataframe with 3 columns
column_names=['Postal Code','Borough','Neighborhood']
df=pd.DataFrame(columns=column_names)

In [6]:
# find the needed postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [7]:
#examine the dataframe result
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#size of dataframe
df.shape

(180, 3)

In [9]:
#if there are any NaN, drop them
df.dropna()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [11]:
#reset the index
df.reset_index(drop=True)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [12]:
#eliminate cells where no boroughs assigned
df2=df[df['Borough']!='Not assigned']

In [13]:
#examine new df
df2.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
#look at size of new df (should be smaller than previous)
df2.shape

(103, 3)

In [15]:
#if a borough has a 'Not assigned' neighborhood, set neighborhood = to borough
df3=df2
df3.loc[df3['Neighborhood']=="Not assigned",'Neighborhood']=df3.loc[df3['Neighborhood']=="Not assigned",'Borough']

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [16]:
#examine new df
df3

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [17]:
#size of new df, should be same size as previous
df3.shape

(103, 3)

In [18]:
#some df tidying, group by postal code
temp_df=df3.groupby('Postal Code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [19]:
#then merge df, key is Postal Code
df_merge = pd.merge(df3, temp_df, on='Postal Code')

In [20]:
#drop extra column and duplicate values
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)

In [22]:
#rename columns in merged set for consistency with previous df
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [23]:
#look at merged df
df_merge.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [62]:
#check size of merged df.  should be same
df_merge.shape

(103, 3)

# Part 2.  Add latitude and longitude of each neighbourhood to data frame

In [24]:
#rename df
df4=df_merge

In [25]:
#importing the lat long from the file provided by Coursera
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [26]:
#view new df
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
#add latitude and longitude columns to the dataframe created in the previous excercise
df4['Latitude'] = geo_df['Latitude']
df4['Longitude'] = geo_df['Longitude']
df4.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476


# Part 3.  Make some maps and do some clustering

In [28]:
#need to insall another library
pip install folium

     |████████████████████████████████| 94 kB 8.2 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [29]:
import folium

In [30]:
#need the lat, long of city of Toronto
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


In [31]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)
    
#map_toronto

In [32]:
#define my Foursquare credentials
CLIENT_ID = '4W2EB5EZAMTUNRAL0RL0PDKESLM1EOP205SZUAFSQTPP4JCS' # your Foursquare ID
CLIENT_SECRET = '1RDOKYAULPXWVJMTA1TJBIE3STCJZ0P1KTMFXIIME15UNQSI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4W2EB5EZAMTUNRAL0RL0PDKESLM1EOP205SZUAFSQTPP4JCS
CLIENT_SECRET:1RDOKYAULPXWVJMTA1TJBIE3STCJZ0P1KTMFXIIME15UNQSI


In [34]:
#Foursquare search for venues, get url for Foursquare query
LIMIT=100
radius =500
#create URL

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=4W2EB5EZAMTUNRAL0RL0PDKESLM1EOP205SZUAFSQTPP4JCS&client_secret=1RDOKYAULPXWVJMTA1TJBIE3STCJZ0P1KTMFXIIME15UNQSI&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'

In [35]:
#run this query
results = requests.get(url).json()
#results

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
#import libraries for JSON files and to make to dataframe
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [38]:
#creates the data fame of venue, category, and lat and long

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()



/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Indigo,Bookstore,43.653515,-79.380696
4,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684


In [39]:
#check the size of the df from venue query
nearby_venues.shape

(74, 4)

In [40]:
#function to query Foursquare for venues and to make a df with neighborhoods and lat long
def getNearbyVenues(names, latitudes, longitudes, radius=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
#run the function
toronto_venues1 = getNearbyVenues(names=df4['Neighborhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [42]:
#look at df
toronto_venues1.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Victoria Village,43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,Victoria Village,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [43]:
#how many venues per neighbourhood?
toronto_venues1.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,15,15,15,15,15,15
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,21,21,21,21,21,21
"Bedford Park, Lawrence Manor East",36,36,36,36,36,36
"Birch Cliff, Cliffside West",99,99,99,99,99,99
...,...,...,...,...,...,...
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,14,14,14,14,14,14
Woodbine Heights,1,1,1,1,1,1


In [44]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues1[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues1['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
totonto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_group = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_group.head()



,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.047619,0.0,0.000000,0.0,0.0,0.047619
3,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.0,0.027778,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.020202,0.0,0.000000,...,0.020202,0.0,0.0,0.0,0.000000,0.0,0.010101,0.0,0.0,0.000000


In [47]:
#function to show most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [49]:
#show most common venues by neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Café,Breakfast Spot,Coffee Shop,Climbing Gym,Grocery Store,Bar,Pet Store,Restaurant,Nightclub,Italian Restaurant
1,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Bridal Shop,Ice Cream Shop,Diner,Sushi Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Sandwich Place
2,Bayview Village,Sandwich Place,Indian Restaurant,Yoga Studio,Discount Store,Burger Joint,Coffee Shop,Pizza Place,Pharmacy,Park,Middle Eastern Restaurant
3,"Bedford Park, Lawrence Manor East",Gastropub,Café,Coffee Shop,Cocktail Bar,Gym,Fountain,Food Truck,Bistro,Camera Store,Middle Eastern Restaurant
4,"Birch Cliff, Cliffside West",Coffee Shop,Café,Hotel,Asian Restaurant,Steakhouse,Japanese Restaurant,Sushi Restaurant,Salad Place,Thai Restaurant,Burrito Place


### Run the clustering algorithms

In [50]:
# set number of clusters
kclusters = 5

toronto_clusters = toronto_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clusters)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1], dtype=int32)

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df4

# merge toronto_group with df4 to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.806686,-79.194353,1.0,Fast Food Restaurant,Yoga Studio,Hobby Shop,Fish & Chips Shop,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
1,M4A,North York,Victoria Village,43.784535,-79.160497,3.0,Home Service,Bar,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,1.0,Mexican Restaurant,Electronics Store,Rental Car Location,Restaurant,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917,1.0,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,1.0,Thai Restaurant,Bank,Caribbean Restaurant,Gas Station,Athletics & Sports,Bakery,Hakka Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [53]:
# add markers to the map and plot
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    try:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)
    except:
        continue
map_clusters

In [54]:
# make a df for stuff in cluster 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,0.0,Gym,Park,Yoga Studio,Discount Store,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
23,East York,0.0,Convenience Store,Park,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
25,Downtown Toronto,0.0,Pool,Food & Drink Shop,Park,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
30,Downtown Toronto,0.0,Park,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
31,West Toronto,0.0,Park,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
48,Downtown Toronto,0.0,Park,Summer Camp,Trail,Yoga Studio,Escape Room,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space
64,York,0.0,Park,Trail,Business Service,Yoga Studio,Distribution Center,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
74,Central Toronto,0.0,Women's Store,Park,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
79,Central Toronto,0.0,Park,Bakery,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
82,Scarborough,0.0,Park,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [55]:
# make a df for stuff in cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Fast Food Restaurant,Yoga Studio,Hobby Shop,Fish & Chips Shop,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
2,Downtown Toronto,1.0,Mexican Restaurant,Electronics Store,Rental Car Location,Restaurant,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
3,North York,1.0,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,Downtown Toronto,1.0,Thai Restaurant,Bank,Caribbean Restaurant,Gas Station,Athletics & Sports,Bakery,Hakka Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,Etobicoke,1.0,Playground,Yoga Studio,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,1.0,Pizza Place,Intersection,Yoga Studio,Discount Store,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
98,Etobicoke,1.0,Jewelry Store,Convenience Store,Yoga Studio,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
99,Downtown Toronto,1.0,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Falafel Restaurant,Event Space,Farmers Market,Ethiopian Restaurant,Diner,Electronics Store
101,Etobicoke,1.0,Pizza Place,Grocery Store,Japanese Restaurant,Fried Chicken Joint,Sandwich Place,Discount Store,Event Space,Ethiopian Restaurant,Falafel Restaurant,Farmers Market


In [56]:
# make a df for stuff in cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,2.0,Baseball Field,Business Service,Yoga Studio,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
91,Downtown Toronto,2.0,Pool,Baseball Field,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
97,Downtown Toronto,2.0,Baseball Field,Yoga Studio,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [57]:
# make a df for stuff in cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,Home Service,Bar,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
63,York,3.0,Home Service,Yoga Studio,Distribution Center,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [58]:
# make a df for stuff in cluster 5

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,East York,4.0,Massage Studio,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


## Observation comments.
For cluster 1, multiple neighbourhoods are covered, the most common venues are mainly recreational.
CLuster 2 has the highest number of venues, across many neighbourhoods.  
Cluster 3 has few venues, mostly recreational facilities.
Cluster 4 has few venues, most common is home service.
Cluster 5 has a single borough.